# Paper

    - Towards identifying distance between opinions
   

In [52]:
import tagme
import os
import pandas as pd
import numpy as np
from tqdm import tqdm


import itertools
import tarfile
from collections import defaultdict

import string

import nltk
nltk.download('punkt')
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

import xlrd


from dotenv import load_dotenv
load_dotenv()
base_dir = '../../data/ssr/'
tagme.GCUBE_TOKEN = os.getenv("GCUBE_TOKEN")

[nltk_data] Downloading package punkt to /home/tusharm/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Random sampling 10K tweets

In [74]:
import random
tweet_ids = set()
while len(tweet_ids) <10000:
    tweet_ids.add(random.randrange(0,annotation_df.shape[0]))

In [75]:
# sampled tweets
sampled_df = annotation_df[annotation_df.index.isin(list(tweet_ids))]


In [76]:
sampled_df.head()

,User ID,Tweet ID,Tweet,Hashtags,tagged users,Language,username
0,775702352523296770,5f1879402d4d197ddb3f7d88,We want justice for is my support in this fig...,"#Candle4SSR,#candle4sushant,","['ishkarnBHANDARI', 'lostsoul_apu', 'justice_s...",en,NaN
3,1209664517220659200,5f1879412d4d197ddb3f7e10,"""Let's get mix more people with us for this fi...","#Candle4SSR,#Candle4SSR,",[],en,NaN
7,1206274223980703744,5f1879442d4d197ddb3f7ef7,Sirjee \nI think you are more than enough for...,"#Candle4SSR,","['ishkarnBHANDARI', 'Official_Arnab_']",en,NaN
19,1284574640799543297,5f1879482d4d197ddb3f8061,#Candle4SSR\nWhy only along with the people...,"#Candle4SSR,","['KanganaTeam', 'ishkarnBHANDARI', 'Swamy39', ...",en,NaN
21,1266220705609248768,5f1879492d4d197ddb3f80a1,The person who says that Committed suicide I ...,"#cbiforsushant,#Candle4SSR,","['itsSSR', 'Republic_Bharat']",en,NaN


In [77]:
sampled_df.to_csv('sampled_10K_tweets.csv', index = False)

# TagMe API

In [37]:
lunch_annotations = tagme.annotate("My favourite meal is Mexican burritos.")
for ann in lunch_annotations.get_annotations(0.1):
    print(ann)

meal -> Meal (score: 0.2014230340719223)
Mexican -> Mexican cuisine (score: 0.36614900827407837)
burritos -> Burrito (score: 0.28607892990112305)


In [4]:
tomatoes_mentions = tagme.mentions("I definitely like ice cream better than tomatoes.")

for mention in tomatoes_mentions.mentions:
    print(mention)

ice cream [18,27] lp=0.18749085068702698
tomatoes [40,48] lp=0.004235605709254742


In [9]:
rels = tagme.relatedness_title(("Barack Obama", "Italy"))
print("Obama and italy have a semantic relation of", rels.relatedness[0].rel)

Obama and italy have a semantic relation of 0.05192309617996216


In [30]:
pre_processed_tweets = pd.read_pickle(f'{base_dir}pre-processed-ssr-tweets.pkl')
pre_processed_tweets.head()

,created_at,User_ID,tweet_text,retweeted_from,retweeted_tweet,ID,tweets,tidy_tweets,tidy_tweet_tokens,tokens_no_stop
0,Wed Jul 22 17:37:02 +0000 2020,1251870540454912002,"b""RT @Isha34576744: We will win..\n\nStay toge...",1.285989e+18,"b""We will win..\n\nStay together and support e...",5f1879402d4d197ddb3f7d59,"b""RT @Isha34576744: We will win..\n\nStay toge...",will stay together support each other sach par...,"[will, stay, together, support, each, other, s...","[stay, together, support, sach, pareshan, sakt..."
1,Wed Jul 22 17:37:02 +0000 2020,1258970340,b'RT @KuttuRobin1: We will make it World Recor...,1.285970e+18,b'We will make it World Record..!!\n\nWe are w...,5f1879402d4d197ddb3f7d5a,b'RT @KuttuRobin1: We will make it World Recor...,will make world record with,"[will, make, world, record, with]","[make, world, record]"
2,Wed Jul 22 17:37:02 +0000 2020,1276202098816716800,b'RT @DhamiKour: #Candle4SSR we want justice f...,1.285989e+18,b'#Candle4SSR we want justice for SSR because ...,5f1879402d4d197ddb3f7d5c,b'RT @DhamiKour: #Candle4SSR we want justice f...,want justice because this murderer want justic...,"[want, justice, because, this, murderer, want,...","[want, justice, murderer, want, justice, inquiry]"
3,Wed Jul 22 17:37:02 +0000 2020,1162601829085016065,b'RT @kushal02yadav: #Candle4SSR\nFair and tra...,1.285969e+18,b'#Candle4SSR\nFair and transparent investigat...,5f1879402d4d197ddb3f7d5e,b'RT @kushal02yadav: #Candle4SSR\nFair and tra...,transparent investigation needed case request ...,"[transparent, investigation, needed, case, req...","[transparent, investigation, needed, case, req..."
4,Wed Jul 22 17:37:02 +0000 2020,2758419409,b'RT @republic: #JusticeforSSR | Kangana Ranau...,1.285987e+18,"b""#JusticeforSSR | Kangana Ranaut has sent a m...",5f1879402d4d197ddb3f7d5f,b'RT @republic: #JusticeforSSR | Kangana Ranau...,kangana ranaut sent message mumbai police sayi...,"[kangana, ranaut, sent, message, mumbai, polic...","[kangana, ranaut, sent, message, mumbai, polic..."


In [33]:
annotation_df = pd.read_csv(f'{base_dir}ssr_annotation_language.csv')
annotation_df.head()

,User ID,Tweet ID,Tweet,Hashtags,tagged users,Language,username
0,775702352523296770,5f1879402d4d197ddb3f7d88,We want justice for is my support in this fig...,"#Candle4SSR,#candle4sushant,","['ishkarnBHANDARI', 'lostsoul_apu', 'justice_s...",en,NaN
1,1082351503354908672,5f1879412d4d197ddb3f7dc2,Nhi raha ek kamyab insan #Candle4SSR,"#Candle4SSR,",[],so,NaN
2,1234061913073618945,5f1879412d4d197ddb3f7e09,#Candle4SSR\n1 million for our real hero.,"#Candle4SSR,",[],en,NaN
3,1209664517220659200,5f1879412d4d197ddb3f7e10,"""Let's get mix more people with us for this fi...","#Candle4SSR,#Candle4SSR,",[],en,NaN
4,1026777124844306432,5f1879422d4d197ddb3f7e1f,Thanks for being with us to seek justice! \nN...,"#Candle4SSR,","['AkshayeOfficial', 'ishkarnBHANDARI']",en,NaN


In [ ]:
subjects_per_tweet = {}
for i,r in tqdm(annotation_df.iterrows()):
    lunch_annotations = tagme.annotate(r['Tweet'])
    mentions = lunch_annotations.get_annotations(0.2)
    subjects_per_tweet[r['Tweet ID']] = []
    for ann in mentions:
        temp = {}
        temp['begin'] = ann.begin # starting index of subject
        temp['end'] = ann.end # ending index of subject
        temp['entity_title'] = ann.entity_title # wikipedia broader name
        temp['mention'] = ann.mention # subject identified
        temp['score'] = ann.score # opinion score
        subjects_per_tweet[r['Tweet ID']].append(temp)
    if len(subjects_per_tweet[r['Tweet ID']])==0:
        subjects_per_tweet.pop(r['Tweet ID'])

4598it [1:11:07,  1.10it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

11955it [3:04:22,  1.09it/s]

In [47]:
subjects_per_tweet

{'5f1879422d4d197ddb3f7e1f': [{'begin': 34,
   'end': 41,
   'entity_title': 'Justice (band)',
   'mention': 'justice',
   'score': 0.23570901155471802}],
 '5f1879422d4d197ddb3f7e3d': [{'begin': 31,
   'end': 39,
   'entity_title': 'Nepotism',
   'mention': 'nepotism',
   'score': 0.24871794879436493}],
 '5f1879432d4d197ddb3f7e9a': [{'begin': 29,
   'end': 38,
   'entity_title': 'Bollywood',
   'mention': 'Bollywood',
   'score': 0.41177576780319214}],
 '5f1879442d4d197ddb3f7ef7': [{'begin': 164,
   'end': 173,
   'entity_title': 'Bollywood',
   'mention': 'Bollywood',
   'score': 0.3659946322441101}],
 '5f1879442d4d197ddb3f7f08': [{'begin': 16,
   'end': 20,
   'entity_title': 'Network interface controller',
   'mention': 'nPar',
   'score': 0.3571428656578064},
  {'begin': 66,
   'end': 70,
   'entity_title': 'Odin',
   'mention': 'harr',
   'score': 0.3143066465854645},
  {'begin': 111,
   'end': 114,
   'entity_title': 'Fokker XB-8',
   'mention': 'xb8',
   'score': 0.3333333432674

# Clustering mentions with entity_titles

In [48]:
mentions_to_titles = {}
for i in subjects_per_tweet.keys():
    l = subjects_per_tweet[i]
    for j in l:
        mentions_to_titles[j['mention']] = j['entity_title']
mentions_to_titles

{'justice': 'Justice',
 'nepotism': 'Nepotism',
 'Bollywood': 'Bollywood',
 'nPar': 'Network interface controller',
 'harr': 'Odin',
 'xb8': 'Fokker XB-8',
 'Sushant': 'Rajan Sushant',
 'soul': 'Soul',
 'rest in peace': 'Rest in peace',
 'Rhea Chakraborty': 'Rhea Chakraborty',
 'Sushant Singh Rajput': 'Sushant Singh Rajput',
 'Tweet': 'Twitter',
 'retweet': 'Twitter',
 'tweet': 'Twitter',
 'can': 'Can (band)',
 'mafia': 'Mafia (1993 film)',
 'Arabian  sea': 'Arabian Sea',
 'Brother': 'Brother (Pearl Jam song)',
 "let's go All The Way": "Let's Go All the Way (song)",
 'a word': 'The A Word',
 'nWe': 'Yin Yin Nwe',
 'Rajput': 'Rajput',
 'mumbai': 'Chhatrapati Shivaji Terminus',
 'mumbai police': 'Mumbai Police',
 'xaf': 'Central African CFA franc',
 'We the people': 'We the People (petitioning system)',
 'suriya': 'Suriya',
 'xb2': 'Xbox 360',
 'bollywood': 'Bollywood',
 'middle class': 'Middle class',
 'But not': 'For but not with',
 'Long long way to go': 'Long, Long Way to Go',
 'arna

In [ ]:
replaced_tweets = []
for i,r in annotation_df.iterrows():
    tweet_words = r['Tweet'].strip().split(' ')
    for j in tweet_words:
        try:
            pass
            

# Lexicons

In [49]:
unigram_df = pd.read_csv('LEXICON_UG.txt', sep = " ")
unigram_df['sentiment'] = np.where(unigram_df['SENTIMENT_SCORE'] > 0, 1, 0)  # 1 is positive, 0 is negative
print(unigram_df.head())

# Create dict with unigram and sentiment
unigram_sentiment_dict = pd.Series(unigram_df.SENTIMENT_SCORE.values,
                                   index=unigram_df.UNIGRAM.values).to_dict()


bigrams_df = pd.read_csv( 'LEXICON_BG.txt', sep=" ")

# Add sentiment column
bigrams_df['sentiment'] = np.where(bigrams_df['SENTIMENT_SCORE'] > 0, 1, 0)  # 1 is positive, 0 is negative
bigrams_df.head()

bigram_sentiment_dict = pd.Series(
    bigrams_df.SENTIMENT_SCORE.values,
    index=bigrams_df.BIGRAM.str.split('-').apply(
        lambda l: tuple(l)
    )
).to_dict()

  UNIGRAM  SENTIMENT_SCORE  sentiment
0      aa         0.019674          1
1     aaa         0.032775          1
2    aaas         0.074593          1
3  aachen         0.011926          1
4     aah         0.118070          1


### Reading adjective classes

In [60]:
xls_file = pd.ExcelFile('ADJECTIVES.xlsx', engine = 'openpyxl')
# print(xls_file.head())

adjective_expansion = pd.read_excel(xls_file, 'ADJECTIVE_EXPANSION').dropna(how='all').reset_index(drop=True)
high_low_PN = pd.read_excel(xls_file, '(HIGH,LOW)_POS_NEG', header=None)[0].values.tolist()
high_low_NP = pd.read_excel(xls_file, '(HIGH,LOW)_NEG_POS', header=None)[0].values.tolist()
fast_slow_PN = pd.read_excel(xls_file, '(FAST,SLOW)_POS_NEG', header=None)[0].values.tolist()
fast_slow_NP = pd.read_excel(xls_file, '(FAST,SLOW)_NEG_POS', header=None)[0].values.tolist()


def clean_adjective_df(df):
    adjectives = []
    for i in range(4):
        adjectives.extend(df.iloc[:, i+1].dropna().tolist())
    
    adj_category = df.iloc[0,0]
    
    return [(adj, adj_category) for adj in adjectives]


tokens = []
token_rows = 5
for i in range(0, len(adjective_expansion), token_rows+1):
    tokens.append(clean_adjective_df(adjective_expansion.loc[i:i+token_rows]))
high_tokens, low_tokens, fast_tokens, slow_tokens = tokens

adjective_class_map = dict(high_tokens + low_tokens + fast_tokens + slow_tokens)

### Reading composition classes

In [62]:
semantic_classes_file = pd.ExcelFile('SEMANTIC_CLASSES.xlsx', engine = 'openpyxl')

dominator_neg = pd.read_excel(semantic_classes_file, 'DOMINATOR_NEG', header=None)[0].values.tolist()
dominator_pos = pd.read_excel(semantic_classes_file, 'DOMINATOR_POS', header=None)[0].values.tolist()
propagator_pos = pd.read_excel(semantic_classes_file, 'PROPAGATOR_POS', header=None)[0].values.tolist()
propagator_neg = pd.read_excel(semantic_classes_file, 'PROPAGATOR_NEG', header=None)[0].values.tolist()
reverser_pos = pd.read_excel(semantic_classes_file, 'REVERSER_POS', header=None)[0].values.tolist()
reverser_neg = pd.read_excel(semantic_classes_file, 'REVERSER_NEG', header=None)[0].values.tolist()

### Calculate combined sentiment

In [69]:
class Adjective:
    FAST = 'FAST_TOKENS'
    SLOW = 'SLOW_TOKENS'
    HIGH = 'HIGH_TOKENS'
    LOW = 'LOW_TOKENS'
    
class Sign:
    POSITIVE = 'Positive'
    NEGATIVE = 'Negative'
    NEUTRAL = 'Neutral'

adjective_conditions = {
    Adjective.FAST: [(fast_slow_PN, Sign.POSITIVE), (fast_slow_NP, Sign.NEGATIVE)],
    Adjective.SLOW: [(fast_slow_PN, Sign.NEGATIVE), (fast_slow_NP, Sign.POSITIVE)],
    Adjective.HIGH: [(high_low_PN, Sign.POSITIVE), (high_low_NP, Sign.NEGATIVE)],
    Adjective.LOW: [(high_low_PN, Sign.NEGATIVE), (high_low_NP, Sign.POSITIVE)],
}

sentiment_to_score = {
    Sign.POSITIVE: +1,
    Sign.NEGATIVE: -1,
}

def is_given_sentiment(sentiment, word, sentiment_map):
    if word in sentiment_map:
        if sentiment==Sign.NEGATIVE and sentiment_map[word] < 0:
            return True
        elif sentiment==Sign.POSITIVE and sentiment_map[word] > 0:
            return True

def calculate_composition_or_adj_sentiment(bigram):
    # Adjective
    adjective_token = adjective_class_map.get(bigram[0])
    if adjective_token is not None:
        for expansions_list, sentiment_sign in adjective_conditions[adjective_token]:
            if bigram[1] in expansions_list:
                return sentiment_to_score[sentiment_sign]

    # Composition: Reverser
    elif bigram[0] in reverser_pos and is_given_sentiment(Sign.NEGATIVE, bigram[1], unigram_sentiment_dict):
        return sentiment_to_score[Sign.POSITIVE]
    elif bigram[0] in reverser_neg and is_given_sentiment(Sign.POSITIVE, bigram[1], unigram_sentiment_dict):
        return sentiment_to_score[Sign.NEGATIVE]

    # Composition: Propagator
    elif bigram[0] in propagator_pos and is_given_sentiment(Sign.NEGATIVE, bigram[0], unigram_sentiment_dict) and is_given_sentiment(Sign.POSITIVE, bigram[1], unigram_sentiment_dict):
        return sentiment_to_score[Sign.POSITIVE]
    elif bigram[0] in propagator_neg and is_given_sentiment(Sign.POSITIVE, bigram[0], unigram_sentiment_dict) and is_given_sentiment(Sign.NEGATIVE, bigram[1], unigram_sentiment_dict):
        return sentiment_to_score[Sign.NEGATIVE]

    # Composition: Dominator
    elif bigram[0] in dominator_neg:
        return sentiment_to_score[Sign.NEGATIVE]
    elif bigram[0] in dominator_pos:
        return sentiment_to_score[Sign.POSITIVE]
    
    return None


In [86]:
def calculate_unigram_sentiment(tokenized_sentence, sentiment_map=unigram_sentiment_dict):
    sentiment_score = 0
    for token in tokenized_sentence:
        token_sentiment = sentiment_map.get(token)
        if token_sentiment is None:
            continue
        else:
            sentiment_score += token_sentiment

    return sentiment_score

def calculate_bigram_sentiment(bigram):
    bigram_sentiment = bigram_sentiment_dict.get(bigram)
    if bigram_sentiment:
        # -0.02 and 0.02 allows for margin of error for neutrals
        if bigram_sentiment < -0.02:
            return -1
        elif bigram_sentiment > 0.02:
            return 1
        else:
            return 0
    return None


## TODO : punish the sentiment with distance from the subject entity before adding to current_sentiment
def calculate_sentiment_combined(sentence, tweet_id):
    
    table = str.maketrans(dict.fromkeys(string.punctuation))
    cleaned_sentence = sentence.translate(table)  # remove punctuation
    sentence_bigrams = list(
        nltk.bigrams(
            word_tokenize(
                cleaned_sentence.lower()
            )
        )
    )
    total_sentiment_score = 0.0
    try:
        for subject in subjects_per_tweet[tweet_id]:
            sentiment_score = 0
            p = 0.0
            n = 0.0
            for bigram in sentence_bigrams:
                current_sentiment = calculate_bigram_sentiment(bigram)
                if current_sentiment is None:
                    current_sentiment = calculate_composition_or_adj_sentiment(bigram)
                    if current_sentiment is None:
                        unigram_sentiment = calculate_unigram_sentiment(bigram)
                        if unigram_sentiment < - 0.1:
                            current_sentiment = -1
                        elif unigram_sentiment > 0.1:
                            current_sentiment = 1
                        else:
                            current_sentiment = 0
                # get the index of the bigram in the sentence
                ind = sentence.index(bigram)
                # calculate the distance between subject and bigram
                dis = min(
                    abs(ind - subject['begin']), 
                    abs(ind - subject['end']), 
                    abs(ind + len(bigram) - subject['begin'])
                )
                # punish the sentiment of a bigram for an opinion subject 
                if (current_sentiment/sqrt(dis)) < (-0.01):
                    n += (current_sentiment/sqrt(dis))
                elif (current_sentiment/sqrt(dis)) > 0.01:
                    p += (current_sentiment/sqrt(dis))
            # sentiment for a given subject 
            sentiment_score +=  (p+n)/(p-n+1)
            # update the polarity of subject
            subject['polarity'] = sentiment_score
            # update the score of the whole sentence
            total_sentiment_score += sentiment_score
        # take the average based on score with respect to different opnion subjects
        total_sentiment_score/= len(subjects_per_tweet)
    except:
        total_sentiment_score = 0.0
        for bigram in sentence_bigrams:
            current_sentiment = calculate_bigram_sentiment(bigram)
            if current_sentiment is None:
                current_sentiment = calculate_composition_or_adj_sentiment(bigram)
                if current_sentiment is None:
                    unigram_sentiment = calculate_unigram_sentiment(bigram)
                    if unigram_sentiment < - 0.1:
                        current_sentiment = -1
                    elif unigram_sentiment > 0.1:
                        current_sentiment = 1
                    else:
                        current_sentiment = 0
            total_sentiment_score += current_sentiment
    
    return total_sentiment_score

In [87]:
score_of_tweet = []
for i,r in tqdm(annotation_df.iterrows()):
    score = calculate_sentiment_combined(r['Tweet'], r['Tweet ID'])
    score_of_tweet.append(score)

49404it [00:40, 1209.65it/s]


In [88]:
annotation_df['Scores'] = score_of_tweet

In [92]:
annotation_df.head()

,User ID,Tweet ID,Tweet,Hashtags,tagged users,Language,username,Scores
0,775702352523296770,5f1879402d4d197ddb3f7d88,We want justice for is my support in this fig...,"#Candle4SSR,#candle4sushant,","['ishkarnBHANDARI', 'lostsoul_apu', 'justice_s...",en,NaN,6.0
1,1082351503354908672,5f1879412d4d197ddb3f7dc2,Nhi raha ek kamyab insan #Candle4SSR,"#Candle4SSR,",[],so,NaN,0.0
2,1234061913073618945,5f1879412d4d197ddb3f7e09,#Candle4SSR\n1 million for our real hero.,"#Candle4SSR,",[],en,NaN,1.0
3,1209664517220659200,5f1879412d4d197ddb3f7e10,"""Let's get mix more people with us for this fi...","#Candle4SSR,#Candle4SSR,",[],en,NaN,4.0
4,1026777124844306432,5f1879422d4d197ddb3f7e1f,Thanks for being with us to seek justice! \nN...,"#Candle4SSR,","['AkshayeOfficial', 'ishkarnBHANDARI']",en,NaN,5.0


# Calculating opnion distances

In [120]:
n = annotation_df.shape[0]
opinion_distance_matrix = []
for i in tqdm(range(n)):
    opinion_distance_matrix.append([])
    for j in range(n):
        opinion_distance_matrix[i].append(0)

100%|██████████| 49404/49404 [04:34<00:00, 179.99it/s]


In [125]:
print(opinion_distance_matrix)
print(len(opinion_distance_matrix[0]))

KeyboardInterrupt: 

In [ ]:

for i,r in tqdm(annotation_df.iterrows()):
    for j,k in annotation_df.iterrows():
        if subjects_per_tweet.get(r['Tweet ID']) and subjects_per_tweet.get(k['Tweet ID']):
            subjects_list1 = subjects_per_tweet[r['Tweet ID']]
            subjects_list2 = subjects_per_tweet[k['Tweet ID']]
            entity_titles1 = set(map(lambda x: x['entity_title'], subjects_list1))
            entity_titles2 = set(map(lambda x: x['entity_title'], subjects_list2))
            common_subjects = entity_titles1 - entity_titles2
            if len(common_subjects)>0:
                od = 0.0
                try:
                    polarities1 =  [x['polarity'] for x in subjects_list1 if x['entity_title'] in common_subjects]
                    polarities2 =  [x['polarity'] for x in subjects_list2 if x['entity_title'] in common_subjects]
                    m = len(polarities1) * len(polarities2)
                    for p1 in polarities1:
                        for p2 in polarities2:
                            od+= abs(p1-p2)
                    opinion_distance_matrix[i][j] = od/(2*m)
                except:
                    opinion_distance_matrix[i][j] = -3
            else:
                opinion_distance_matrix[i][j] = -3
        else:
            opinion_distance_matrix[i][j] = -3

8it [00:33,  4.34s/it]

In [128]:
opinion_distance_matrix[0][1] = 0
opinion_distance_matrix[4][:10]

[-3, -3, -3, -3, -3, 0, 0, 0, 0, 0]